In [ ]:
# hide 
#default_exp cli
#default_cls_lvl 3
from nbdev import *
from drt.cli import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export
#hide
from drt.environment import DataIntakeEnv
from drt.create_environment import create_environment
from drt.registration import register_all
from drt.verification import check_datagroup
from drt.receipt import sync_data_folder
import drt.data_model as dm
from pathlib import Path
import argparse
import sys
import typer 
import types

app = typer.Typer()

# Command line functions

> Below the functions used to call the applicaitons are defined. Functions are grouped into a few main areas, `create`, `register`, `verify`, `document`, `clean` these different areas allow you to perform actions in those domains.

In [ ]:
#export
@app.command()
def create(location:str ='.'):
    """
    create a new data registration area
    """
    if not location or location == '.':
        config_file = create_environment(Path.cwd())
    elif location[0] == '/':
        config_file = create_environment(Path(location))
    else:
        config_file = create_environment((Path.cwd() / location))

In [ ]:
#export
@app.command()
def register(config_file:Path = '.config/config.ini', force:bool = False):
    """
    register any new data elements, run with --force to regeister existing data elements
    """
    env = DataIntakeEnv(config_file)
    if force:
        if input(
                "!!! WARNING --force is a DESTRUCTIVE action, system information for specified dataset will be overwritten.\nIf no data groups are specified that means ALL data groups.\n\nAre you sure you want to continue? (Y/N)") == "Y":
            print("Activating forced recalculation")
            env.force_recalculate = True
        else:
            print("Proceeding without force, registering new data only.")

    # TODO add data group and data group type specific registration calls

    register_all(env)

In [ ]:
#export
@app.command()
def verify(config_file:Path = ".config/config.ini", calc_hash:bool = False):
    """
    verify the data registration environment to determine if there are data elements which need remediating. 
    i.e. there is data missing, a report missing, or a script missing. 
    this will also check the meta data of the files in the file system against the recorded metadata. If your
    data isn't too big, run this with --calc_hash to activate hash verification.
    """
    env = DataIntakeEnv(config_file)
    if calc_hash:
        [check_datagroup(env, group_type, light=False) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]
    else:
        [check_datagroup(env, group_type) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]




In [ ]:
#export
@app.command()
def document(config_file:Path = './config/config.ini', force:bool = False, sync:bool =True):
    """
    Build documentation based on information in db and provided by analysts
    """
    env = DataIntakeEnv(config_file)
    if force:
        if input(
                "!!! WARNING --force is a DESTRUCTIVE action, documentation will be overwritten.\nIf no data groups are specified that means ALL data groups.\n\nAre you sure you want to continue? (Y/N)") == "Y":
            print("Activating forced documentation rebuild")
            env.force_rebuild = True
        else:
            print("Proceeding without force, synchronising data and rebuilding.")
    if sync:
        [sync_data_folder(env, group_type) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]
    

In [ ]:
#export
def main():
    app()

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_cli.ipynb.
Converted 02_create_environment.ipynb.
Converted 03_data_model.ipynb.
Converted 04_environment.ipynb.
Converted 05_receipt.ipynb.
Converted 06_registration.ipynb.
Converted 07_verification.ipynb.
Converted 99_utils.ipynb.
Converted index.ipynb.
